# Task 1: Data Exploration and Enrichment

This notebook explores the Ethiopia Financial Inclusion dataset and documents the enrichment process.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path().resolve().parent / 'src'))

from data_loader import load_unified_data, load_reference_codes

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [ ]:
# Load original data
main_data, impact_links = load_unified_data()
ref_codes = load_reference_codes()

print(f"Main data shape: {main_data.shape}")
print(f"Impact links shape: {impact_links.shape}")
print(f"Reference codes shape: {ref_codes.shape}")